# SpaceHackathon

In [2]:
import os.path
videosets_path = os.path.abspath('tests\\step1')    # путь к папке с видеосетами
videoset = 1 # номер видеосета

### Обработка Excel-таблицы с настройками камер

In [ ]:
from excel import parse_camera_settings

excel_read_path = videosets_path + f'\\videoset{videoset}\\Seq{videoset}_settings.xlsx'

cd = parse_camera_settings(excel_read_path, videoset)
camera_data = list(cd.values())

### Сбор видеофайлов и подготовка к обработке

In [4]:
from video_processing import VideoProcessor

video_processors = []
video_processors_t = []

for i in range(3):
    video_path = videosets_path + f'\\videoset{videoset}\\Seq{videoset}_camera{i+1}.mov'
    video_path_t = videosets_path + f'\\videoset{videoset}\\Seq{videoset}_camera{i+1}T.mov'
    video_processors.append(VideoProcessor(video_path, camera_data[i], 'rgb'))
    video_processors_t.append(VideoProcessor(video_path_t, camera_data[i], 'ir'))

### Обработка видеофайлов

In [5]:
from concurrent.futures import ThreadPoolExecutor

coords_from_rgb = []
coords_from_ir = []

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(lambda vp: vp.get_all_coords(), vp) for i, vp in enumerate(video_processors)]
    for future in futures:
        coords_from_rgb.append(future.result())

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(lambda vp: vp.get_all_coords(), vp) for i, vp in enumerate(video_processors_t)]
    for future in futures:
        coords_from_ir.append(future.result())

### Отображение результатов обработки видеофайлов

In [7]:
import pandas as pd
df1 = pd.DataFrame([data if data is not None else  {'x':float('nan'),'y':float('nan'),'diameter':float('nan')} for data in coords_from_rgb[0]])
df2 = pd.DataFrame([data if data is not None else  {'x':float('nan'),'y':float('nan'),'diameter':float('nan')} for data in coords_from_rgb[1]])
df3 = pd.DataFrame([data if data is not None else  {'x':float('nan'),'y':float('nan'),'diameter':float('nan')} for data in coords_from_rgb[2]])
df2

,x,y,diameter
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
237,NaN,NaN,NaN
238,NaN,NaN,NaN
239,NaN,NaN,NaN
240,NaN,NaN,NaN


### TODO: Сделать анализ данных и сохранение результатов